# Confiabilidade de viga mista

## Importando bibliotecas

In [366]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import import_ipynb
from math import sqrt
import pandas as pd

## Tabela de perfis

In [367]:
tabela = pd.read_excel('Catalogo-Tabela-Bitolas-Editado-2.xlsx')
display(tabela)

,massa_linear,d,bf,tw,tf,h,d',area,Ix,Wx,...,ly,Wy,ry,Zy,rt,lt,Mesa,Alma,Cw,u\n
0,13.0,148,100,4.3,4.9,138,118,16.6,635,85.8,...,82,16.4,2.22,25.5,2.60,1.72,10.20,27.49,4181,0.67
1,18.0,153,102,5.8,7.1,139,119,23.4,939,122.8,...,126,24.7,2.32,38.5,2.69,4.34,7.18,20.48,6683,0.69
2,22.5,152,152,5.8,6.6,139,119,29.0,1229,161.7,...,387,50.9,3.65,77.9,4.10,4.75,11.52,20.48,20417,0.88
3,24.0,160,102,6.6,10.3,139,115,31.5,1384,173.0,...,183,35.9,2.41,55.8,2.73,11.08,4.95,17.48,10206,0.69
4,29.8,157,153,6.6,9.3,138,118,38.5,1739,221.5,...,556,72.6,3.80,110.8,4.18,10.95,8.23,17.94,30277,0.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,154.2,623,229,14.0,24.9,573,541,196.5,125783,4038.0,...,4999,436.6,5.04,683.3,5.94,303.29,4.60,38.66,4456995,2.11
104,155.0,611,324,12.7,19.0,573,541,198.1,129583,4241.7,...,10783,665.6,7.38,1022.6,8.53,200.77,8.53,42.60,9436714,2.47
105,174.0,616,325,14.0,21.6,573,541,222.8,147754,4797.2,...,12374,761.5,7.45,1171.1,8.58,286.88,7.52,38.63,10915665,2.48
106,195.0,622,327,15.4,24.4,573,541,250.1,168484,5417.5,...,14240,870.9,7.55,"1341 ,0",8.66,405.29,6.70,35.14,12695302,2.49


## Definindo funções

In [368]:
def calculo_destribuicao_media(media: float, desvpad: float, area: bool = False) -> list:
    AMOSTRAS: int = 1000
    medias: list = []
    areas: list = []
    
    if (area):
        for i in range(AMOSTRAS):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            areas.append(np.pi * (medias[i] / 2) ** 2)
        
        return medias, areas

    else:
        for i in range(AMOSTRAS):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            
        return medias
        

## Médias - Espessura da laje

In [369]:
espessura_laje: float = 10 # cm 
desvpad_espessura_laje: float = 0.06 * espessura_laje # cm

espessuras_laje: list = calculo_destribuicao_media(espessura_laje, desvpad_espessura_laje)

## Médias - Limite de escoamento do perfil de aço

In [370]:
limite_escoamento_perfil: float = 250 * 1.08 # MPa
desvpad_escoamento_perfil: float = 0.08 * limite_escoamento_perfil # MPa

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_perfil, desvpad_escoamento_perfil)

## Médias - Limite de escoamento da armadura

In [371]:
limite_escoamento_armadura: float = 500 * 1.08 # MPa
desvpad_escoamento_armadura: float = 0.05 * limite_escoamento_armadura #MPa

limites_escoamento_armadura: list = calculo_destribuicao_media(limite_escoamento_armadura, desvpad_escoamento_armadura)

## Médias - Resistência do concreto à compressão

In [372]:
resistencia_concreto: float = 20 * 1.17 # MPa
desvpad_resistencia_concreto: float = 0.15 * resistencia_concreto # MPa

resistencias_concreto: list = calculo_destribuicao_media(resistencia_concreto, desvpad_resistencia_concreto)

## Médias - Diamêtro do rebite

In [373]:
diametro_rebite: float = 1.59 # cm
desvpad_diametro_rebite: float = 0.01 * diametro_rebite # cm

diametros_rebites, areas_rebites = calculo_destribuicao_media(diametro_rebite, desvpad_diametro_rebite, True)


## Médias - Resistência do rebite

In [374]:
resistencia_rebite: float = 415 * 1.06 # MPa
desvpad_resistencia_rebite: float = 0.06 * resistencia_rebite # MPa

resistencias_rebite: list = calculo_destribuicao_media(resistencia_rebite, desvpad_resistencia_rebite)

## Pré-dimensionamento

In [375]:
carga_permanente_g1: float = 7.6 # kN/m
carga_permanente_g2: float = 5 # kN/m

variacao_carga: list = [3, 11]
carga_utilizacao_q2: list = [i for i in range(variacao_carga[0], variacao_carga[1])]

carga_construcao_q1: list = [i / 5.6 for i in carga_utilizacao_q2]

# Propriedades da laje
vao_laje: float = 9 # m
espacamento_laje: float = 2.8 # m

# Carga distrubuída do projeto
carga_distribuida_projeto: list = [1.4 * (carga_permanente_g1 + carga_permanente_g2) + carga_construcao_q1[i] * carga_utilizacao_q2[i] for i in range(len(carga_utilizacao_q2))]

# Momento solicitante do projeto
momento_solicitante_projeto: list = [(carga_distribuida_projeto[i] * (vao_laje ** 2)) / 8 for i in range(len(carga_distribuida_projeto))]

altura_perfil_ht: float = 45 # cm
coeficiente_ya1: float = 1.1
x: float = 10 # cm

# Tabela com áreas filtradas a partir de 450 mm
tabela_filtrada = tabela.loc[tabela['d'] >= 450]

row_tabela: int = 0

# Área mínima da seção de aço
areas_minimas: list = [[] for i in range(len(momento_solicitante_projeto))]
index_matriz: int = 0
area_minima_inicial: float = tabela_filtrada.iloc[0]['area']
for i in momento_solicitante_projeto:
    for j in range(len(limites_escoamento_perfil)):
        areas_minimas[index_matriz].append((i * 100) / (((limites_escoamento_perfil[j] / 10) / coeficiente_ya1) * ((altura_perfil_ht / 2) + espessuras_laje[j] - (x / 2)))) # cm²
        if (areas_minimas[index_matriz][j] > area_minima_inicial):
            print(f'Área mínima inicial: {area_minima_inicial}')
            print(f'Área mínima calculada: {areas_minimas[index_matriz][j]}')
            row_tabela += 1
            area_minima_inicial = tabela_filtrada.iloc[row_tabela]['area']
    index_matriz += 1
    

print(f'Áreas mínimas: {areas_minimas}')
print(f'Área mínima final: {area_minima_inicial}')


Área mínima inicial: 66.6
Área mínima calculada: 70.14027745221121
Áreas mínimas: [[29.32457766967599, 27.559192028144498, 32.12550651062042, 32.635097593860415, 28.659365004079255, 28.753033458856155, 28.443465186459484, 29.25822038027482, 29.4057994639151, 27.47496505230436, 35.16468617641893, 28.42054726463304, 30.351729312273957, 25.966770230444702, 29.392173350697167, 28.571800220622947, 25.78137576254567, 26.684256607448592, 25.40712048436701, 27.368123988612346, 31.278755467053788, 30.263305571688058, 31.836841899251077, 29.669662735298726, 29.199090235583007, 27.529588329029274, 28.94053522914569, 27.47411666203532, 29.883040365840273, 28.304525418488257, 27.38370116011275, 35.07202992470624, 27.66484868401645, 29.4460420051945, 28.114707710636424, 27.77794285046778, 27.112510826731537, 29.51800803633507, 30.698818405631155, 31.61394575129784, 26.842883241870275, 30.82131322348419, 30.237981439453062, 26.984185175278824, 32.21600719740203, 27.627819165485093, 28.528035118271568

## Flambagem da seção

In [376]:
# Classificação da seção quanto a flambagem
altura_perfil: float = tabela_filtrada.iloc[row_tabela]['d\''] # mm
altura_total_perfil: float = tabela_filtrada.iloc[row_tabela]['d'] # mm
espessura_perfil: float = tabela_filtrada.iloc[row_tabela]['tw'] # mm
modulo_elasticidade_perfil: float = 200000 # MPa
print(f'Altura perfil: {altura_perfil}, Altura total do perfil: {altura_total_perfil} e Espessura do perfil: {espessura_perfil}')

esbeltez_alma: float = altura_perfil / espessura_perfil

limites_flambagem_local: list = []

def calculo_limite_flambagem_local(coef: float, modulo_elasticidade_perfil: float, limite_escoamento_perfil: float) -> float:
    return coef * sqrt(modulo_elasticidade_perfil / limite_escoamento_perfil)

contador_compacto: int = 0
contador_semicompacto: int = 0

for i in range(len(limites_escoamento_perfil)):
    limite = calculo_limite_flambagem_local(3.76, modulo_elasticidade_perfil, limites_escoamento_perfil[i])
    limites_flambagem_local.append(limite)
    
    if esbeltez_alma <= limite:
        contador_compacto += 1
    elif esbeltez_alma > limite and esbeltez_alma < calculo_limite_flambagem_local(5.7, modulo_elasticidade_perfil, limites_escoamento_perfil[i]):
        contador_semicompacto += 1
    else:
        print('Erro')
        
print(f'Compactas: {contador_compacto}')
print(f'Semi-compactas: {contador_semicompacto}')
        

Altura perfil: 404, Altura total do perfil: 455 e Espessura do perfil: 8.0
Compactas: 1000
Semi-compactas: 0


## Momento resistência da viga mista

In [377]:
# Resistência à compressão do concreto

yc: float = 1.4
largura_efetiva_laje: float = 225 # cm

rcd: list = []

for i in range(len(resistencias_concreto)):
    rcd.append(0.85 * (resistencias_concreto[i] / 10) * largura_efetiva_laje * espessuras_laje[i] / yc)

print(f'Rcd: {rcd}')

# Resistência à tração do aço

rtd: list = []

for i in range(len(limites_escoamento_perfil)):
    rtd.append(area_minima_inicial * (limites_escoamento_perfil[i] / 10) / coeficiente_ya1)
    
print(f'Rtd: {rtd}')

profundidades_linha_neutra: list = []
momentos_resistente_laje_concreto: list = []

for i in range(len(rtd)):
    if rcd[i] >= rtd[i]:
        aux = rtd[i]/(rcd[i] / espessuras_laje[i])
        profundidades_linha_neutra.append(aux)
        momentos_resistente_laje_concreto.append(rtd[i] * ((altura_total_perfil / 20) + espessuras_laje[i] - aux / 2) / 100)
        
print(f'x: {profundidades_linha_neutra}')
print(f'Momentos resistentes da laje de concreto:\n{momentos_resistente_laje_concreto}')


Rcd: [3358.9591048414186, 2683.5345875203116, 2543.7102249939016, 3307.221390484316, 3637.871676514649, 2706.423661246719, 4581.041912915016, 2948.4642274624, 2498.0649587788394, 3052.1844486523073, 2676.32286602689, 1965.100357580893, 2854.5255056966007, 2968.0079023242415, 3664.003403870252, 2954.7440254000985, 4558.253394049238, 3961.691879149386, 3847.176876647643, 2077.1559498698625, 3605.537479869038, 3372.9740971899787, 3815.7198156611666, 2886.5976877548615, 3366.7155419715164, 2402.84835370447, 3426.7266594436423, 2782.810064864474, 2746.4916299248466, 3542.1413616696927, 3810.1997847649827, 2760.3409626715043, 2736.407551110342, 3443.9973017706916, 3190.8424807324486, 3500.0961141859893, 3473.9791315270845, 3935.727330865673, 3177.5434944086137, 2582.089347492437, 3175.7719332276138, 2590.116787982722, 3905.788718983103, 4405.610803728127, 2590.323558800902, 2605.771685908501, 3447.771774942239, 3585.59734061571, 2859.413495923842, 3872.848498774742, 3087.9572813367736, 2817.

## Confiabilidade momento resistente